參考網址[[link]](https://medium.com/%E8%8A%B1%E5%93%A5%E7%9A%84%E5%A5%87%E5%B9%BB%E6%97%85%E7%A8%8B/geocoding-%E6%89%B9%E9%87%8F%E8%99%95%E7%90%86%E5%9C%B0%E5%9D%80%E8%BD%89%E6%8F%9B%E7%B6%93%E7%B7%AF%E5%BA%A6-721ab2564c88)

# Define a Function

In [2]:
import requests
import time
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from bs4 import BeautifulSoup
from selenium.webdriver.support.ui import WebDriverWait 
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from msedge.selenium_tools import EdgeOptions
from msedge.selenium_tools import Edge
import random

# for Chrome (無介面執行)
options = webdriver.ChromeOptions()
options.add_argument("headless")

# for Edge (無介面執行) (https://www.cnblogs.com/lixuelong/p/14451608.html)
edge_options = EdgeOptions()
edge_options.use_chromium = True
# # 设置无界面模式，也可以添加其它设置
edge_options.add_argument('headless')


In [3]:
# addr 地址
# chromedriver.exe or msedgedriver.exe 的路徑

# for Chrome
# path = 'C:\Program Files (x86)\Python-3.6.11\Python-3.6.11\chromedriver.exe'

# for Edge
path = 'C:\Program Files (x86)\Python-3.6.11\Python-3.6.11\msedgedriver.exe'

In [4]:
def get_coordinate(addr):
    # for Chrome 

    # browser = webdriver.Chrome(executable_path = path,options=options) 
        
    # for Edge (https://www.cnblogs.com/lixuelong/p/14451608.html)     
    # browser = webdriver.Edge(executable_path = path, options=edge_options )  # 
    browser = Edge(executable_path = path, options=edge_options)
    browser.get("http://www.map.com.tw/") 
    wait = WebDriverWait(browser, 30)
    time.sleep(random.randint(0,2))  
      
    #找到輸入地址的地方並且清空    
    wait.until(EC.presence_of_element_located((By.XPATH,"//*[@id='searchWord']")))
    # time.sleep(random.randint(1,2)) 
    search = browser.find_element_by_xpath("//*[@id='searchWord']")
    search.clear()

    # 輸入地址 
    time.sleep(random.randint(1,2))
    search.send_keys(addr)
    
    # 按下搜尋！
    wait.until(EC.element_to_be_clickable((By.XPATH, "//body/form[@id='form1']/div[@id='Menu']/div[2]/img[2]"))).click()
    
    # browser.find_element_by_xpath("//body/form[@id='form1']/div[@id='Menu']/div[2]/img[2]").click()   
        
    # 轉到跳出來的框框(用 xpath 比較好喔！)
    time.sleep(random.randint(2,3))
    wait.until(EC.presence_of_element_located((By.XPATH, "//*[@id='customMarkinfowindow']/div/iframe")))
    iframe = browser.find_element_by_xpath("//*[@id='customMarkinfowindow']/div/iframe")
    wait.until(EC.frame_to_be_available_and_switch_to_it(iframe)) #這個會自動轉
    # browser.switch_to.frame(iframe)
    
    # 取得完整地址
    time.sleep(random.randint(1,2))
    full_addr = wait.until(EC.presence_of_element_located((By.XPATH, "//*[@id='defaultInfo']/table/tbody/tr[1]/td/table/tbody/tr[4]/td"))).text
    # full_addr = browser.find_element_by_xpath("//*[@id='defaultInfo']/table/tbody/t[1]/td/table/tbody/tr[4]/td")
    # print(full_addr)

    # 按下'座標'這個按鈕
    time.sleep(random.randint(1,2))
    wait.until(EC.element_to_be_clickable((By.XPATH, "//*[@id='defaultInfo']/table/tbody/tr[3]/td/table/tbody/tr/td[2]"))).click()
    # browser.find_element_by_xpath("//*[@id='defaultInfo']/table/tbody/tr[3]/td/table/tbody/tr/td[2]").click()
    
    # 擷取經緯度
    coor = browser.find_element_by_xpath("//*[@id='location']/table/tbody/tr[2]/td")
    coor = coor.text.strip().split(" ")
    lat = coor[-1].split("：")[-1]
    log = coor[0].split("：")[-1]
    browser.quit()    
    return [full_addr, lat, log]

# 讀入表格

In [5]:
import pandas as pd
import numpy as np
from collections import Counter

讀資料

In [7]:

path_market = '..\..\Roaming-Dogs_data\Location_data\Temple\宗教團體查詢結果.csv'
dt = pd.read_csv(path_market)
# Counter(dt['縣市']).most_common()
print(dt.shape)



(11842, 8)


轉經緯度(拜託讓我轉！)

In [7]:
lon_lat = []
print(dt.shape[0])

for i in range(dt.shape[0]):

    print(i)    
    print('-----------')        
    address = dt['地址.1'][i]
    x = get_coordinate(address)
    lon_lat.append(x) 
    time.sleep(random.randint(10,15))

1421
0
-----------


C:\Users\CHENWE~1\AppData\Local\Temp/ipykernel_4368/274035846.py:8: DeprecationWarning: Selenium Tools for Microsoft Edge is deprecated. Please upgrade to Selenium 4 which has built-in support for Microsoft Edge (Chromium): https://docs.microsoft.com/en-us/microsoft-edge/webdriver-chromium/#upgrading-from-selenium-3
  browser = Edge(executable_path = path, options=edge_options)


1
-----------
2
-----------
3
-----------
4
-----------
5
-----------
6
-----------
7
-----------
8
-----------
9
-----------
10
-----------
11
-----------
12
-----------
13
-----------
14
-----------
15
-----------
16
-----------


In [ ]:
lon_lat_df = pd.DataFrame(lon_lat)
lon_lat_df.columns = ['完整地址', '經度', '緯度']
print(lon_lat_df.shape)

In [ ]:
# encoding 記得轉，不然是亂碼！
lon_lat_df.to_csv(f'..\..\Roaming-Dogs_data\Location_data\Temple\寺廟_{c}_經緯度_01.csv', encoding='big5', index = None)